In [ ]:
!pip install ultralytics

In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files
import os, shutil, json
from pathlib import Path
import numpy as np

files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"nadyana120876","key":"f8464b9a119178d1ac11edd17e44167a"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d nadyana120876/dataset-with-coco
!unzip dataset-with-coco.zip -d /content/import-dataset

Streaming output truncated to the last 5000 lines.
  inflating: /content/import-dataset/dataset/images/valid/661621d4a14e487c8c74354134a29710.jpg  
  inflating: /content/import-dataset/dataset/images/valid/661bef3ef00b4bb7b29e797ea4db91c2.jpg  
  inflating: /content/import-dataset/dataset/images/valid/662ab83988d14ebfaf059b2980cad943.jpg  
  inflating: /content/import-dataset/dataset/images/valid/662e821b90574df08e8f21ecaea3a6c7.jpg  
  inflating: /content/import-dataset/dataset/images/valid/6638ad392fb843119c3a2b626efb6c1f.jpg  
  inflating: /content/import-dataset/dataset/images/valid/6638ede8fcba41deaf2f44c77da8cc82.jpg  
  inflating: /content/import-dataset/dataset/images/valid/664367a2a1a04e31acc9df46d10cbb65.jpg  
  inflating: /content/import-dataset/dataset/images/valid/664a10815646427cb123b7f194d01714.jpg  
  inflating: /content/import-dataset/dataset/images/valid/666107a01af8430c8c347e7699908912.jpg  
  inflating: /content/import-dataset/dataset/images/valid/666f837887f740a982

In [ ]:
dataset_root = "/content/dataset"
os.makedirs(f"{dataset_root}/images", exist_ok=True)
os.makedirs(f"{dataset_root}/labels/train", exist_ok=True)
os.makedirs(f"{dataset_root}/labels/valid", exist_ok=True)

og_train_dir = "/content/import-dataset/dataset/images/train"
shutil.move(og_train_dir, f"{dataset_root}/images")

og_val_dir = "/content/import-dataset/dataset/images/valid"
shutil.move(og_val_dir, f"{dataset_root}/images")


Error: Destination path '/content/dataset/images/train' already exists

In [ ]:
og_test_dir = "/content/import-dataset/dataset/images/test"
shutil.move(og_test_dir, f"{dataset_root}/images")

'/content/dataset/images/test'

In [ ]:
os.makedirs(f"{dataset_root}/annotations", exist_ok=True)
shutil.move("/content/import-dataset/json/train.json", f"{dataset_root}/annotations/train.json")
shutil.move("/content/import-dataset/json/val.json", f"{dataset_root}/annotations/valid.json")

'/content/dataset/annotations/valid.json'

In [ ]:
shutil.move("/content/import-dataset/json/test.json", f"{dataset_root}/annotations/test.json")

'/content/dataset/annotations/test.json'

In [ ]:
from pycocotools.coco import COCO
import os
import json

def convert_coco_to_yolo(coco_json_path, images_dir, labels_dir):
    coco = COCO(coco_json_path)
    os.makedirs(labels_dir, exist_ok=True)

    # Create a mapping (category_id → class index starting at 0)
    categories = coco.loadCats(coco.getCatIds())
    category_map = {cat['id']: i for i, cat in enumerate(categories)}

    for img_id in coco.imgs:
        img_info = coco.loadImgs(img_id)[0]
        file_name = img_info['file_name']
        width, height = img_info['width'], img_info['height']

        # Create a txt label file for each image
        label_file = os.path.join(labels_dir, file_name).replace('.jpg', '.txt').replace('.png', '.txt')
        with open(label_file, 'w') as f:
            ann_ids = coco.getAnnIds(imgIds=img_id)
            anns = coco.loadAnns(ann_ids)
            if len(anns) == 0:
              print(f"No annotations for image: {file_name}")
              continue

            for ann in anns:
                x, y, w, h = ann['bbox']  # COCO format: top-left x,y,width,height
                x_center = (x + w / 2) / width
                y_center = (y + h / 2) / height
                w /= width
                h /= height

                class_id = category_map[ann['category_id']]
                f.write(f"{class_id} {x_center} {y_center} {w} {h}\n")

    print(f"✅ Done! Labels saved in: {labels_dir}")

# --- RUN FOR TRAIN SET ---
convert_coco_to_yolo(
    coco_json_path="/content/dataset/annotations/train.json",
    images_dir="/content/dataset/images/train",
    labels_dir="/content/dataset/labels/train"
)

# --- RUN FOR VAL SET ---
convert_coco_to_yolo(
    coco_json_path="/content/dataset/annotations/valid.json",
    images_dir="/content/dataset/images/valid",
    labels_dir="/content/dataset/labels/valid"
)


loading annotations into memory...
Done (t=0.18s)
creating index...
index created!
✅ Done! Labels saved in: /content/dataset/labels/train
loading annotations into memory...
Done (t=0.04s)
creating index...
index created!
✅ Done! Labels saved in: /content/dataset/labels/valid


In [ ]:
convert_coco_to_yolo(
    coco_json_path="/content/dataset/annotations/test.json",
    images_dir="/content/dataset/images/test",
    labels_dir="/content/dataset/labels/test"
)

loading annotations into memory...
Done (t=0.04s)
creating index...
index created!
✅ Done! Labels saved in: /content/dataset/labels/test


In [ ]:
import yaml

coco_json_path = "/content/dataset/annotations/train.json"
with open(coco_json_path, 'r') as f:
  data = json.load(f)

categories = data['categories']
class_names = [cat['name'] for cat in categories]

yaml_data = {
    "train": "/content/dataset/images/train",
    "val": "/content/dataset/images/valid",
    "test": "/content/dataset/images/train",
    "nc": len(class_names),
    "names": class_names
}

yaml_path = "/content/dataset/data.yaml"
with open(yaml_path, 'w') as f:
    yaml.dump(yaml_data, f)

In [ ]:
from ultralytics import YOLO
yaml_dir = "/content/dataset/data.yaml"

model = YOLO("yolov10m.pt")
model.train(data=yaml_dir, epochs=30, imgsz=640, batch=24, lr0=0.001)

Ultralytics 8.3.218 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=24, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov10m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plot

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 295,  296,  297,  298,  299,  313,  314,  315,  316,  317,  318,  319,  320,  321,  322,  323,  324,  325,  326,  327,  328,  329,  330,  331,  332,  333,  334,  335,  336,  337,  338,  339,  340,  341,  342,  343,  344,  345,  346,  347,  348,  349,  350,  351,  352,  353,  354,  355,  356,  357,  358,  359,
        360,  361,  362,  363,  364,  365,  366,  367,  368,  369,  370,  371,  372,  373,  374,  375,  376,  377,  378,  379,  380,  381,  382,  392,  393,  394,  395,  396,  397,  398,  399,  400,  401,  402,  446,  447,  448,  449,  450,  451,  452,  453,  454,  455,  456,  457,  458,  459,  460,  461,  462,  463,
        464,  465,  466,  467,  468,  469,  470,  471,  472,  473,  474,  475,  476,  477,  478,  479,  480,  481,  482,  483,  484,  485,  486,  487,  488,  489,  490,  491,  492,  493,  494,  495,  496,  497,  498,  499,  500,  501,  502,  503,  504,  505,  506,  507,  508,  509,  

In [ ]:
from google.colab import files
files.download('/content/runs/detect/train2/weights/best.pt')
files.download('/content/runs/detect/train2/weights/last.pt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
drive_folder = "/content/drive/MyDrive/YOLO_Training_Backups"
!mkdir -p {drive_folder}
!cp -r /content/runs/detect/train2 {drive_folder}/
!ls {drive_folder}/train2


args.yaml			 results.csv	       val_batch0_labels.jpg
BoxF1_curve.png			 results.png	       val_batch0_pred.jpg
BoxP_curve.png			 train_batch0.jpg      val_batch1_labels.jpg
BoxPR_curve.png			 train_batch1.jpg      val_batch1_pred.jpg
BoxR_curve.png			 train_batch2.jpg      val_batch2_labels.jpg
confusion_matrix_normalized.png  train_batch32260.jpg  val_batch2_pred.jpg
confusion_matrix.png		 train_batch32261.jpg  weights
labels.jpg			 train_batch32262.jpg


Test

In [ ]:
from ultralytics import YOLO
model = YOLO("/content/runs/detect/train2/weights/best.pt")
results = model.val(data="/content/dataset/data.yaml", split='test', imgsz=640, batch=16)

Ultralytics 8.3.218 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
YOLOv10m summary (fused): 136 layers, 15,898,537 parameters, 0 gradients, 62.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2888.6±1290.7 MB/s, size: 134.0 KB)
val: Scanning /content/dataset/labels/train.cache... 38665 images, 24 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 38689/38689 51.1Mit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 2419/2419 10.6it/s 3:49
                   all      38689      38665      0.925      0.914      0.956      0.805
Common Eider (Adult male)         59         59      0.977      0.949      0.975      0.832
Long-tailed Duck (Winter male)         65         65      0.954      0.948      0.983      0.798
Ruddy Duck (Breeding male)         66         66      0.969       0.97      0.993      0.771
Swainson's Hawk (Dark morph )         61         61      0.801      0.852      0.926      

In [ ]:
!cp -r /content/test_results.csv {drive_folder}/

In [ ]:
import pandas as pd
metrics = results.results_dict
df = pd.DataFrame([metrics])
csv_path = "/content/test_results.csv"
df.to_csv(csv_path, index=False)


In [ ]:
# get class-wise metrics
class_names = results.names                       # {0: 'cat', 1: 'dog', ...}
precision = results.box.p                        # per-class precision
recall = results.box.r                           # per-class recall
map50 = results.box.ap50                         # per-class AP@0.5
map5095 = results.box.ap                         # per-class AP@0.5:0.95

# Save to CSV
rows = []

num_classes_available = len(precision)  # valid indexes = 0 to num_classes_available-1

for cls_id, name in class_names.items():
    if cls_id < num_classes_available:   # ✅ avoid index errors
        rows.append({
            "class_id": cls_id,
            "class_name": name,
            "precision": float(precision[cls_id]),
            "recall": float(recall[cls_id]),
            "mAP50": float(map50[cls_id]),
            "mAP50-95": float(map5095[cls_id])
        })
    else:
        # Class did not appear in validation → no metrics available
        rows.append({
            "class_id": cls_id,
            "class_name": name,
            "precision": None,
            "recall": None,
            "mAP50": None,
            "mAP50-95": None
        })

df = pd.DataFrame(rows)
df.to_csv("/content/per_class_metrics.csv", index=False)


In [ ]:
!cp -r /content/runs/detect/train {drive_folder}/